# Yelp Dataset
https://www.yelp.com/dataset/
- unzip the first .tar file, it will extract a single file
- add .tar to the end of the file's name then extract again
- the second file will extract 6 json files
- for the purpose of this project, we would only need business and review json files

In [ ]:
# !unzip -o data.zip

In [2]:
import pandas as pd
pd.set_option('display.max_columns', 500) # Make sure we can see all of the columns
pd.set_option('display.max_rows', 200)

## Load/Clean Business JSON

In [3]:
business = pd.read_json('data/business.json', lines=True)
print(business.shape)
business.head()

(192609, 14)


,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,2818 E Camino Acequia Drive,{'GoodForKids': 'False'},1SWheh84yJXfytovILXOAQ,"Golf, Active Life",Phoenix,None,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0,AZ
1,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC
3,"15655 W Roosevelt St, Ste 237",None,xvX2CttrVhyG2z1dFg_0xw,"Insurance, Financial Services",Goodyear,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,33.455613,-112.395596,Farmers Insurance - Paul Lorenz,85338,3,5.0,AZ
4,"4209 Stuart Andrew Blvd, Ste F","{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...",HhyxOkGAM07SRYtlQ4wMFQ,"Plumbing, Shopping, Local Services, Home Servi...",Charlotte,"{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ...",1,35.190012,-80.887223,Queen City Plumbing,28217,4,4.0,NC


In [4]:
# Keep only business that are still open
business = business[business['is_open']==1]
# Drop columns that aren't relavent
business = business.drop(['address','attributes','hours','latitude','longitude',
                          'postal_code','review_count','name','is_open'], axis=1)
# Remove any data that is NaN
business = business.dropna()
print(business.shape)
business.head()

(158064, 5)


,business_id,categories,city,stars,state
1,QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,2.5,ON
2,gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,4.0,NC
3,xvX2CttrVhyG2z1dFg_0xw,"Insurance, Financial Services",Goodyear,5.0,AZ
4,HhyxOkGAM07SRYtlQ4wMFQ,"Plumbing, Shopping, Local Services, Home Servi...",Charlotte,4.0,NC
5,68dUKd8_8liJ7in4aWOSEA,"Shipping Centers, Couriers & Delivery Services...",Mississauga,2.5,ON


In [13]:
# Without the categories, we cannot group the reviews together
# we would have to remove the null categories
business.isnull().sum()

business_id      0
categories     461
city             0
stars            0
state            0
dtype: int64

In [32]:
business['state'].value_counts().shape

(36,)

In [5]:
business_AZ = business[business['state']=='AZ']
print(business_AZ.shape)
business_AZ.head()

(46754, 5)


,business_id,categories,city,stars,state
3,xvX2CttrVhyG2z1dFg_0xw,"Insurance, Financial Services",Goodyear,5.0,AZ
11,1Dfx3zM-rW4n-31KeC8sJg,"Restaurants, Breakfast & Brunch, Mexican, Taco...",Phoenix,3.0,AZ
16,giC3pVVFxCRR89rApqklyw,"Hair Stylists, Beauty & Spas, Hair Salons, Men...",Scottsdale,5.0,AZ
31,mNBp4KI2goFJKDB9VLGP9w,"Automotive, Auto Repair",Phoenix,5.0,AZ
37,M9DM1ktbW-TB7nXu3Z4RDw,"Home Services, Real Estate, Apartments, Hotels...",Mesa,2.5,AZ


In [6]:
rows_list = []
for x in range(len(business_AZ)):
    cat_list = business_AZ.iloc[x][1].split(', ')
    for y in range(len(cat_list)):
        dict1 = {}
        dict1.update({'business_id':business_AZ.iloc[x][0]})
        dict1.update({'categories':cat_list[y]})
        dict1.update({'city':business_AZ.iloc[x][2]})
        dict1.update({'stars':business_AZ.iloc[x][3]})
        dict1.update({'state':business_AZ.iloc[x][4]})
        rows_list.append(dict1)
business_categories_AZ = pd.DataFrame(rows_list)

In [38]:
rows_list = []
for x in range(len(business)):
    cat_list = business.iloc[x][1].split(', ')
    for y in range(len(cat_list)):
        dict1 = {}
        dict1.update({'business_id':business.iloc[x][0]})
        dict1.update({'categories':cat_list[y]})
        dict1.update({'city':business.iloc[x][2]})
        dict1.update({'stars':business.iloc[x][3]})
        dict1.update({'state':business.iloc[x][4]})
        rows_list.append(dict1)
business_categories = pd.DataFrame(rows_list) 

In [10]:
print(business_categories_AZ.shape)
business_categories_AZ.head()

(201358, 5)


,business_id,categories,city,stars,state
0,xvX2CttrVhyG2z1dFg_0xw,Insurance,Goodyear,5.0,AZ
1,xvX2CttrVhyG2z1dFg_0xw,Financial Services,Goodyear,5.0,AZ
2,1Dfx3zM-rW4n-31KeC8sJg,Restaurants,Phoenix,3.0,AZ
3,1Dfx3zM-rW4n-31KeC8sJg,Breakfast & Brunch,Phoenix,3.0,AZ
4,1Dfx3zM-rW4n-31KeC8sJg,Mexican,Phoenix,3.0,AZ


In [8]:
len(business_categories_AZ['categories'].value_counts())

1203

In [13]:
yelp_AZ_20_category_list = business_categories_AZ['categories'].value_counts()[:20].index
yelp_AZ_20_category_list

Index(['Home Services', 'Shopping', 'Restaurants', 'Health & Medical',
       'Beauty & Spas', 'Local Services', 'Food', 'Automotive', 'Doctors',
       'Active Life', 'Professional Services', 'Event Planning & Services',
       'Real Estate', 'Home & Garden', 'Auto Repair', 'Hair Salons',
       'Fast Food', 'Fashion', 'Nightlife', 'Contractors'],
      dtype='object')

In [15]:
AZ_20_category_df_list = []
for category in yelp_AZ_20_category_list:
    AZ_category = business_categories_AZ[business_categories_AZ['categories']==category]
    AZ_20_category_df_list.append(AZ_category)
len(AZ_20_category_df_list)

20

In [16]:
AZ_20_category_df_list[0].categories.value_counts()

Home Services    8484
Name: categories, dtype: int64

In [24]:
for x in range(10):
    print(str(x+1)+' AZ - Home Services', len(AZ_20_category_df_list))

0 AZ - Home Services 20
1 AZ - Home Services 20
2 AZ - Home Services 20
3 AZ - Home Services 20
4 AZ - Home Services 20
5 AZ - Home Services 20
6 AZ - Home Services 20
7 AZ - Home Services 20
8 AZ - Home Services 20
9 AZ - Home Services 20


In [44]:
business_categories['categories'].value_counts()

Restaurants                        42237
Shopping                           26734
Food                               23208
Home Services                      18455
Beauty & Spas                      16545
Health & Medical                   15875
Local Services                     12830
Automotive                         11955
Nightlife                           9396
Event Planning & Services           8960
Active Life                         8217
Bars                                8155
Fast Food                           6409
Fashion                             6293
Hair Salons                         5919
Professional Services               5844
Sandwiches                          5753
Coffee & Tea                        5731
Home & Garden                       5667
Auto Repair                         5562
Doctors                             5521
Hotels & Travel                     5404
American (Traditional)              5264
Pizza                               5254
Real Estate     

In [40]:
print(business_categories.shape)
business_categories.head()

(658930, 5)


,business_id,categories,city,stars,state
0,QXAEGFB4oINsVuTFxEYKFQ,Specialty Food,Mississauga,2.5,ON
1,QXAEGFB4oINsVuTFxEYKFQ,Restaurants,Mississauga,2.5,ON
2,QXAEGFB4oINsVuTFxEYKFQ,Dim Sum,Mississauga,2.5,ON
3,QXAEGFB4oINsVuTFxEYKFQ,Imported Food,Mississauga,2.5,ON
4,QXAEGFB4oINsVuTFxEYKFQ,Food,Mississauga,2.5,ON


In [17]:
yelp_AZ_20_category_list

Index(['Home Services', 'Shopping', 'Restaurants', 'Health & Medical',
       'Beauty & Spas', 'Local Services', 'Food', 'Automotive', 'Doctors',
       'Active Life', 'Professional Services', 'Event Planning & Services',
       'Real Estate', 'Home & Garden', 'Auto Repair', 'Hair Salons',
       'Fast Food', 'Fashion', 'Nightlife', 'Contractors'],
      dtype='object')

## Create CSV after merging Review and Business JSON

In [ ]:
import pandas as pd

# set chuncksize
size = 1000000
# Iterate over the list of categories(20)
for x in range(20):
    review = pd.read_json('review.json', lines=True,
                          dtype={'review_id':str,'user_id':str,'business_id':str,'stars':int,
                                 'date':str,'text':str,'useful':int,'funny':int,'cool':int},
                          chunksize=size)

    chunk_list = []
    for chunk_review in review:
        chunk_review = chunk_review.drop(['review_id','user_id'], axis=1)
        chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
        chunk_category = pd.merge(AZ_20_category_df_list[x], chunk_review, on='business_id', how='inner')
        chunk_list.append(chunk_category)
        print(x+1, yelp_AZ_20_category_list[x], chunk_category.shape)
    df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)
    df.to_csv(yelp_AZ_20_category_list[x]+".csv", index=False)
    print(df.shape)
    df.head()